
# Accessing APIs

## Introduction

This notebook will guide you through the fundamentals of accessing and working with web APIs using Python. APIs (Application Programming Interfaces) are essential tools for modern data science, allowing you to access real-time data from countless sources including government databases, weather services, financial markets, and research institutions.

We'll work with real-world APIs to gather data about Rwanda, demonstrating practical techniques you can apply to any data science project. You'll learn to handle different types of APIs, from completely free services to those requiring authentication, while following professional best practices.

### Why This Matters

APIs are everywhere in modern data science. Whether you're building economic analysis dashboards, environmental monitoring systems, or social research platforms, the ability to programmatically access external data sources is crucial. This workshop teaches you not just how to make API calls, but how to do so reliably, ethically, and professionally.

## Learning Outcomes

By the end of this workshop, you will be able to:

1. **Make successful API requests** using Python's `requests` library, handling different authentication methods and response formats
2. **Process and analyze JSON data** returned by APIs, converting it into pandas DataFrames for further analysis
3. **Implement professional practices** including error handling, rate limiting, retry logic, and proper credential management
4. **Work with multiple API types** from free government APIs to commercial services requiring API keys
5. **Combine data from multiple sources** to create comprehensive datasets for analysis

---

# Python Setup
Lets make sure we import the required libraries

In [1]:
import os
from dotenv import load_dotenv
import requests
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import Image

In [2]:
load_dotenv()  # take environment variables from .env.

True

# Accessing a Useless API Just to Learn
The [Open Notify API](http://open-notify.org) is an open source project to provide a simple programming interface for some of NASA’s data related
space and spacecrafts. Even though this is a toy example, the process of using APIs is similar for any 
API you will need to access in the future. For this tutorial, the task is to find the latest position of the Internation Space Station(ISS)

## Understanding The API
As mentioned during the introductory lecture, its important to read the API documentation 
and understand the available **endpoints**. Also, to determine the exact endpoint for the data we are interested in getting.
- **Base URL:** For this API, the base url is: ```http://api.open-notify.org```. So, our full url which we will pass to requests will be ```base-url + endpoint```
- **Available Endpoints/Data:** You can check [here](http://open-notify.org/Open-Notify-API/) for avaible endpoints
- **Target Endpoint:** For our task, the endpoint is ```iss-now.json```

## Making the Request
Now, lets request for the data. We create a simple function which makes the requests 
and prints the status code to see if we got a good result.

In [3]:
base_url = 'http://api.open-notify.org/'

In [4]:
# endpoint for current position of ISS
pos_now = 'iss-now.json'

In [5]:
# base_url + endpoint = full url which we can use in requests.
full_url = base_url + pos_now
full_url

'http://api.open-notify.org/iss-now.json'

In [6]:
# Make a request using requests package
response = requests.get(full_url)
print(response.status_code)

200


In [7]:
response.status_code

200

## Use json() to retrieve the json object/data

In [8]:
# Get the data
dict_obj_from_json = response.json()
type(dict_obj_from_json)
print(dict_obj_from_json),type(dict_obj_from_json)

{'message': 'success', 'iss_position': {'longitude': '-1.8734', 'latitude': '-10.1975'}, 'timestamp': 1758116663}


(None, dict)

In [9]:
dict_obj_from_json.keys()

dict_keys(['message', 'iss_position', 'timestamp'])

In [10]:
geo_info = dict_obj_from_json['iss_position']
geo_info

{'longitude': '-1.8734', 'latitude': '-10.1975'}

In [11]:
lon = geo_info['longitude']
print(lon)
lat = geo_info['latitude']
print(lat)

-1.8734
-10.1975


In [12]:
geo_info = dict_obj_from_json['iss_position']
lon = geo_info['longitude']
lat = geo_info['latitude']
print(lat,'--', lon)

-10.1975 -- -1.8734


In [13]:
base_url = 'http://api.open-notify.org/'
#endpoint for current position of ISS
pos_now = 'iss-now.json'
# Full url
full_url = base_url + pos_now


# Make a request using requests package
r = requests.get(full_url)

# Get the data
dict_obj_from_json = r.json()

# Extract Lat/Lon
geo_info = dict_obj_from_json['iss_position']
lon = geo_info['longitude']
lat = geo_info['latitude']
print(lat,'--', lon)

-8.7375 -- -0.7970


In [14]:
def request_iss_position(endpoint_url=None):
    # Make a get request 
    response = requests.get(endpoint_url)

    # Print the status code of the response.
    status_code = response.status_code
    if status_code == 200:
        print('Hooray!!, it worked')
        
        # access the returned data
        dict_obj_from_json = response.json()
        print('Type: {}'.format(type(dict_obj_from_json)))
        print('This is the nested dictionary returned: \n {}'.format(dict_obj_from_json))
        lat = dict_obj_from_json['iss_position']['latitude']
        lon = dict_obj_from_json['iss_position']['longitude']
        print('ISS is passing on Lat: {}, and Lon: {} right about now'.format(lat, lon))
    else:
        print('What happened, lets see the code we got: {}'.format(status_code))

In [15]:
base_url = 'http://api.open-notify.org/'
iss_pos = base_url + 'iss-now.json'
request_iss_position(endpoint_url=iss_pos)

Hooray!!, it worked
Type: <class 'dict'>
This is the nested dictionary returned: 
 {'message': 'success', 'iss_position': {'longitude': '-0.1704', 'latitude': '-7.8798'}, 'timestamp': 1758116709}
ISS is passing on Lat: -7.8798, and Lon: -0.1704 right about now


In [19]:
# Now, try to enter a wrong endpoint and see what happens
iss_pos2 = base_url + 'iss-now.json'
request_iss_position(endpoint_url=iss_pos2)

Hooray!!, it worked
Type: <class 'dict'>
This is the nested dictionary returned: 
 {'message': 'success', 'iss_position': {'longitude': '4.0995', 'latitude': '-1.9257'}, 'timestamp': 1758116827}
ISS is passing on Lat: -1.9257, and Lon: 4.0995 right about now


# Accessing the Data
Now, lets change our function so that we print the Latitude and Longitude of where ISS is right now.

In [20]:
request_iss_position(endpoint_url='http://api.open-notify.org/iss-now.json')
request_iss_position

Hooray!!, it worked
Type: <class 'dict'>
This is the nested dictionary returned: 
 {'message': 'success', 'iss_position': {'longitude': '4.3699', 'latitude': '-1.5447'}, 'timestamp': 1758116834}
ISS is passing on Lat: -1.5447, and Lon: 4.3699 right about now


<function __main__.request_iss_position(endpoint_url=None)>

# EXERCISE-1:
Using the same API, check how many Astronouts are in space right now?

In [27]:
# base-url is same as above
# end point for this data: please check documentation


import requests

# API endpoint for astronauts currently in space
num_astros_url = "http://api.open-notify.org/astros.json"

# Make request
response = requests.get(num_astros_url)

# Check response
if response.status_code == 200:
    # Extract JSON directly (no need for .loads here)
    res = response.json()

    # Print the whole dictionary to inspect
    print("Full response dictionary:")
    print(res)
    print("\nType:", type(res))

    # Extract number of astronauts
    num_people = res.get("number", "Unknown")
    print(f"\nNumber of people in space right now: {num_people}")

    # Extract astronaut details (name and craft)
    people = res.get("people", [])
    print("\nList of astronauts in space:")

    for person in people:
        print(f"- {person['name']} on {person['craft']}")
else:
    print(f"Request failed with status code {response.status_code}")

Full response dictionary:
{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}

Type: <class 'dict'>

Number of people in space right now: 12

List of astronauts in space:
- Oleg Kononenko on ISS
- Nikolai Chub on ISS
- Tracy Caldwell Dyson on ISS
- Matthew Dominick on ISS
- Michael Barratt on ISS
- Jeanette Epps on ISS
- Alexander Grebenkin on ISS
- Butch Wilmore on ISS
- Sunita Williams on ISS
- Li Guangsu on Tiangong
- Li Cong on Tiangong
- Ye Guangfu on Tia

# Accessing the Open Weather API
## About the API
You can read all about it [here](https://openweathermap.org). In summary, they provide weather data for cities across  the world. When you read the API documentation there are several things you need to be aware of and check as follows:
1. How to get credentials to enable access to the API. We only need an API key in this case. Did you get yours?
2. How to make API calls. What type of parameters are required to make and API call to this particular API. In this case, you will note that you will need location.
3. What type of information is available
4. Whats the cost of accessing the data. If free, are there any limits.

## Potential Use Cases
1. **A Weather app on a Smartphone.** Extract data from the API and use it on your app

2. **Historical climate data analysisis for multiple cities.** With climate change being a big thing now, you may want to do analysis comparing weather information across multiple cities.

The API has many versions and subscriptions plans but in our exercise, we will use their 2.5 version which is the one which doesnt require subscription.

## Example Task: Get current weather data and forecasts for a given city or multiple cities
Given a city name, lets get current weather data and forecasts and then save that information as a CSV file. 
We are only interested in the following variables:
- **Weather:** temp, min and max temperature, humidity
- **Other:** Date, with no time

### Approach
We will achieve this by defining three main functions:
1. A small helper function to retrieve city ID from given city name just because using ID is easier than name
2. A function to make thr API call and retrieve the weather data
3. A function to take the returned data, put it into a pandas dataframe.
4. A function putting everything together 

In [28]:
def get_weather(base_url, category='forecast', api_key=None, city_id=None, lat=None, lon=None):
    """
    Use request to make a get request to the open weather API
    :param api_key: This is taken from the API documentation
    :param category: Wether to get current weather, forecast or other type. Check API docs for details.
    :param city_id: ID for the city we need taken from the city.list.json provided by the API
    :param api_key: The API key you got from the API 
    :param lon: Longitude in case we want to use latitude and longitude
    :param lat: Latitude in case we want to use latitude and longitude
    
    :return: A JSON oobject with the data
    """
    # Create full_url based on whether we are using lat/lon, city ID
    if city_id:
        full_url = "{}{}?id={}&APPID={}".format(base_url, category, city_id, api_key)
    
    # use requests to retrieve data from the API
    response = requests.get(full_url)

    # retrieve JSON from the response object
    if response.status_code == 200:
        json_obj = response.json()
    else:
        print('Something went wrong, see error below')
        print(response.status_code)

    # return the JSON object
    return json_obj

In [29]:
def city_id_from_name(city_file, name):
    """
    Retrieve city ID given city name
    :param city_file: JSON file with list of city names and corresponding IDs provided by open weather API
    :param name: Name of city which we want to retrieve ID for
    :return: An integer representing city ID
    """
    # Load JSON file
    fopen = open(city_file, encoding="utf8")
    city_list = json.load(fopen)
    
    # Get city ID which matches with city name
    for c in city_list:
        if c['name'] == name:
            return c['id'] 

In [30]:
def weather_json_to_pandas_df(weather_json):
    """
    Converts the JSON with weather data into a pandas Dataframe
    :param weather_json:The JSON object with weather information
    :return: A pandas Dataframe
    """
    
    # using method for accessing a dictionary
    # put weather items in a list
    weather_items = weather_json['list']
    
    # save into a dataframe
    data = []  # will hold our data
    
    for i in weather_items:
        # get forecast time and truncate to date only
        date_str = i['dt_txt'][:10]
    
        # get temperature, rain and humidity
        temp = i['main']['temp']
        temp_min = i['main']['temp_min']
        temp_max = i['main']['temp_max']
        hum = i['main']['humidity']
    
        data_item = {'Date': date_str, 'tempF': temp,'tempF_min': temp_min, 
                     'tempF_max': temp_min, 'humidity': hum}
    
        # append to list of create earlier on
        data.append(data_item)
    

    # create dataframe
    df = pd.DataFrame(data)
    
    return df

In [31]:
def save_weather_data(city_name, csv_filename, city_list_file):
    """
    Puts everything together and saves weather into CSV file
    :param city_name: Target city name
    :param csv_filename: Full path of CSV filen to save data
    :param city_list_file: File containing city names and IDs
    :return: Saves data into CSV
    """
    # ==============================
    # GET WEATHER DATA
    # ==============================
    # Retrieve city ID and get data
    cityid = city_id_from_name(city_list_file, city_name)
    print('City-ID for {} is {}'.format(city_name, cityid))
    
    weather_data = get_weather(base_url=BASE_URL, api_key=API_KEY,
                           city_id=cityid)
    
    # Put data into Pandas Dataframe
    df = weather_json_to_pandas_df(weather_json=weather_data)
    
    # Add city name 
    df['City'] = city_name
    print()
    print('First few rows of the data')
    print(df.head())
    
    # Save data into CSV
    df.to_csv(city_name, index=False)
    
    print()
    print('Successfully Saved CSV file')
    print("============================")
    
    return df

## Simple Request on Weather API Using City Name
- ```BASE_URL: https://api.openweathermap.org/data/2.5/```
### Parameters:
- version: 2.5 for free plans
- category: weather 
    - weather (current weather by city)
    - forecast (up to 16 days forecast)
- Location (city name, lat/lon, city id)
- API key

In [32]:
# ========================================================
# HARD CODE BASE INFORMAITON AND USE THEM AS GLOBAL VARS
# ========================================================
# Harcode base URL, we use the 2.5 version which seems to be the free version with no need to subscribe
BASE_URL = "https://api.openweathermap.org/data/2.5/"

API_KEY = os.getenv("OPENWEATHER_API_KEY")

# Create full URL for current weather in Kigali
category = 'weather'
city_name = "Kigali"
full_url = "{}{}?q={}&APPID={}".format(BASE_URL, category, city_name, API_KEY)
print(full_url)


# Send request
response = requests.get(full_url)
print("Status:", response.status_code)
print("Body:", response.text)  # if 401, body text is usually explicit


https://api.openweathermap.org/data/2.5/weather?q=Kigali&APPID=4fcad110a72a5c7cbe4924532276b757
Status: 200
Body: {"coord":{"lon":30.0588,"lat":-1.95},"weather":[{"id":500,"main":"Rain","description":"light rain","icon":"10d"}],"base":"stations","main":{"temp":299.86,"feels_like":300.14,"temp_min":299.86,"temp_max":299.86,"pressure":1011,"humidity":47,"sea_level":1011,"grnd_level":848},"visibility":10000,"wind":{"speed":3.6,"deg":160},"rain":{"1h":0.77},"clouds":{"all":40},"dt":1758117433,"sys":{"type":1,"id":2594,"country":"RW","sunrise":1758081081,"sunset":1758124644},"timezone":7200,"id":202061,"name":"Kigali","cod":200}


In [33]:
weather_today_kigali = response.json()
print(weather_today_kigali)

{'coord': {'lon': 30.0588, 'lat': -1.95}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 299.86, 'feels_like': 300.14, 'temp_min': 299.86, 'temp_max': 299.86, 'pressure': 1011, 'humidity': 47, 'sea_level': 1011, 'grnd_level': 848}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 160}, 'rain': {'1h': 0.77}, 'clouds': {'all': 40}, 'dt': 1758117433, 'sys': {'type': 1, 'id': 2594, 'country': 'RW', 'sunrise': 1758081081, 'sunset': 1758124644}, 'timezone': 7200, 'id': 202061, 'name': 'Kigali', 'cod': 200}


In [34]:
weather_today_kigali['main']['temp']  # temperature in Kelvin


299.86

In [35]:
# Get temperature in Kelvin from the weather data
temp_kelvin = weather_today_kigali['main']['temp']

# Convert to Celsius (K - 273.15)
temp_celsius = temp_kelvin - 273.15

print(f"Temperature in Kigali:")
print(f"Kelvin: {temp_kelvin:.2f}K")
print(f"Celsius: {temp_celsius:.2f}°C")

Temperature in Kigali:
Kelvin: 299.86K
Celsius: 26.71°C


In [36]:
weather_today_kigali['dt']


1758117433

In [38]:
from datetime import datetime

# Get timestamp from weather data
timestamp = weather_today_kigali['dt']

# Convert to datetime object and format
readable_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
print(f"Weather data timestamp: {readable_time}")

Weather data timestamp: 2025-09-17 15:57:13


In [39]:
weather_today_kigali.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'rain', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

## Use Functions to Get 16 Days Forecast

In [ ]:
# =======================================
# GET WEATHER FORECAST DATA FOR LILONGWE
# =======================================
city = 'Kigali'
city_json = "/Users/dmatekenya/My Drive (dmatekenya@gmail.com)/TEACHING/AIMS-DSCBI/data/city.list.json"
out_csv = "LL-weather.csv"
df_weather = save_weather_data(city_name=city, csv_filename=out_csv, city_list_file=city_json)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dmatekenya/My Drive (dmatekenya@gmail.com)/TEACHING/AIMS-DSCBI/data/city.list.json'

In [41]:
df_weather

NameError: name 'df_weather' is not defined

## EXERCISE-1: Instead of using city ID, lets use latitude and longitude to get the same information.

### Task description:
- Modify the ```get_weather()``` so that it takes in latitude and longitude and returns today's temperature in degress Celsius.
- To test your function, get current weather data for 'lon': 30.1289, 'lat': -1.9695 and print out the **temperature, place name and country**. The result will look like screenshot below.

### Tips:
- Go to the API documentation to find out how to use lat and lon in the API call
- Also, use the API documentation to check how to change the units 

# https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}